In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tracker import *
import os

In [3]:
def motionDetection():
    cap = cv.VideoCapture('sample.mp4')
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()
    
    tracker = EuclideanDistTracker()
    
    object_detector = cv.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

    while cap.isOpened():
        mask1 = object_detector.apply(frame1)
        mask2 = object_detector.apply(frame2)

        diff = cv.absdiff(frame1, frame2)
        diff_gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)
        blur = cv.GaussianBlur(diff_gray, (5, 5), 0)
        _, thresh = cv.threshold(blur, 20, 255, cv.THRESH_BINARY)
        dilated = cv.dilate(thresh, None, iterations=3)
        contours, _ = cv.findContours(dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        
        detection = []
        for contour in contours:
            if cv.contourArea(contour) < 900:
                continue
            (x, y, w, h) = cv.boundingRect(contour)
            detection.append([x, y, w, h])
            # cv.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
        # Object Tracking
        boxes_ids = tracker.update(detection)
        for box_id in boxes_ids:
            x, y, w, h, id = box_id
            cv.putText(frame1, str(id), (x, y - 15), cv.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
            cv.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 3)

        cv.imshow('Video', frame1)
        frame1 = frame2
        ret, frame2 = cap.read()
        
        if cv.waitKey(50) == 27:
            break
            
    cap.release()
    cv.destroyAllWindows()

In [65]:
# motionDetection()

(720, 960)
(720, 960)
{0: (225, 477), 1: (136, 422), 2: (355, 651)}
{0: (225, 477), 1: (136, 427), 2: (355, 651)}
(720, 960)
{2: (339, 639), 0: (225, 477), 1: (136, 427)}
{2: (339, 639), 0: (223, 480), 1: (136, 427)}
{2: (339, 639), 0: (223, 480), 1: (127, 431)}
(720, 960)
{2: (331, 643), 0: (223, 480), 1: (127, 431)}
{2: (331, 643), 0: (223, 468), 1: (127, 431)}
{2: (331, 643), 0: (223, 468), 1: (116, 425), 3: (182, 426)}
(720, 960)
{2: (331, 643), 0: (217, 481), 3: (182, 426), 1: (116, 425), 4: (271, 410), 5: (44, 234), 6: (248, 224)}
{2: (331, 643), 0: (217, 481), 3: (182, 426), 1: (120, 428), 4: (271, 410), 5: (44, 234), 6: (248, 224)}
{2: (331, 643), 0: (217, 481), 3: (182, 426), 1: (120, 428), 4: (271, 410), 5: (48, 235), 6: (248, 224)}
(720, 960)
{0: (217, 481), 1: (130, 423), 5: (48, 235)}
{0: (217, 481), 1: (130, 423), 5: (42, 251)}
(720, 960)
{1: (113, 427), 5: (42, 251), 7: (244, 204)}
{1: (113, 427), 5: (42, 245), 7: (244, 204)}
(720, 960)
{1: (113, 427), 5: (42, 245), 8: (

## Save images in specific frame

In [10]:
# dir_path = 'images'
# file_name = 'image'

# cap = cv.VideoCapture('sample.mp4')

# # Make sure the 'images' directory exists
# os.makedirs(dir_path, exist_ok=True)

# i = 0
# frame_idx = 0
# while cap.isOpened():
#     ret, frame = cap.read()
    
#     if not ret:
#         break
    
#     if frame_idx < 567 and frame_idx > 400 or frame_idx > 800 and frame_idx < 967:
#         path = os.path.join(dir_path, f'{file_name}{i:04d}.png')
#         cv.imwrite(path, frame)
#         i += 1
#     frame_idx += 1

# cap.release()

## Crop image horizontally half

In [4]:
def crop_images(input_path, output_path, middle_point):
    if not os.path.exists(input_path):
        print('Directory does not exist')
        return
    
    files = os.listdir(input_path)
    for i, file in enumerate(files):
        # read images
        i_path = os.path.join(input_path, file)
        image = cv.imread(i_path)

        # Image process
        img1, img2 = crop_image(image, middle_point)

        # save images
        os.makedirs(output_path, exist_ok=True)
        o_path1 = os.path.join(output_path, f'image{i}_up.jpg')
        o_path2 = os.path.join(output_path, f'image{i}_down.jpg')
        cv.imwrite(o_path1, img1)
        cv.imwrite(o_path2, img2)

def crop_image(image, middle_point):
    # Image process
    # height = image.shape[0] // 2
    img1 = image[0: middle_point, :, :]
    img2 = image[middle_point:, :, :]
    img2 = cv.rotate(img2, cv.ROTATE_180)

    return img1, img2

# dir_path = 'images'
# crop_images(dir_path, 'crop_images', 315)

## Fisheye frame undistorted
Cannot get the correct intrinsic matrix and distortion matrix

In [8]:
# # Load a frame from your fisheye camera
# frame = cv.imread('images\image0030.png')
# if frame is None:
#     print('frame is not loaded')
    
# # Fisheye camera calibration parameters
# K = np.array([[556.20835637, 0, 256.88266933],
#               [0, 556.11754229, 259.93875071],
#               [0, 0, 1]])

# D = np.array([-0.13280831, 0.12408884, -0.00057422, 0.00046171, -0.07185642])

# # Undistort the frame
# undistorted_frame = cv.undistort(frame, K, D)

# cv.imshow('Undistorted frame', undistorted_frame)
# cv.imshow('frame', frame)

# cv.waitKey(0)
# cv.destroyAllWindows()

## Transform image with warpPerspective

In [5]:
def undistort_frame(input):
    left_pts1 = np.float32([[126, 130], [514, 42], [126, 593], [514, 687]])
    left_pts2 =np.float32([[0, 0], [400, 0], [0, 640], [400, 640]])

    right_pts1 = np.float32([[514, 42], [730, 96], [514, 687], [730, 640]])
    right_pts2 = np.float32([[0, 0], [200, 0], [0, 640], [200, 640]])

    left_M = cv.getPerspectiveTransform(left_pts1, left_pts2)
    right_M = cv.getPerspectiveTransform(right_pts1, right_pts2)

    # input = cv.imread('images\image0001.png')
    left_out = cv.warpPerspective(input, left_M, (400, 640))
    right_out = cv.warpPerspective(input, right_M, (200, 640))

    # Merge two images
    output = np.hstack((left_out, right_out))
    # output = cv.resize(output, (input.shape[1], input.shape[0]), interpolation=cv.INTER_AREA)
    
    # cv.imshow('Left Output', left_out)
    # cv.imshow('Right Output', right_out)
    # cv.imshow('Output', output)

    # cv.waitKey(0)
    # cv.destroyAllWindows()
    return output

## Modelscope model
human head prediction

In [33]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

model_id = 'damo/cv_tinynas_head-detection_damoyolo'
input_location = 'https://modelscope.oss-cn-beijing.aliyuncs.com/test/images/image_detection.jpg'

head_detection = pipeline(Tasks.domain_specific_object_detection, model=model_id)
result = head_detection(input_location)
print("result is : ", result)

2023-11-02 11:45:38,386 - modelscope - WARNING - Model revision not specified, use revision: v1.1.1
2023-11-02 11:45:38,972 - modelscope - INFO - initiate model from C:\Users\junee\.cache\modelscope\hub\damo\cv_tinynas_head-detection_damoyolo
2023-11-02 11:45:38,972 - modelscope - INFO - initiate model from location C:\Users\junee\.cache\modelscope\hub\damo\cv_tinynas_head-detection_damoyolo.
2023-11-02 11:45:38,972 - modelscope - INFO - initialize model from C:\Users\junee\.cache\modelscope\hub\damo\cv_tinynas_head-detection_damoyolo
2023-11-02 11:45:40,758 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-11-02 11:45:40,758 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'train': {'mosaic_mixup': {'mosaic_prob': 1.0, 'mosaic_scale': [0.1, 2.0], 'mosaic_size': [640, 640], 'mixup_prob': 0.15, 'mixup_scale': [0.5, 1.5], 'degrees': 10.0, 'translate': 0.2, 'shear': 

result is :  {'scores': array([0.83934075, 0.8387197 , 0.81702626], dtype=float32), 'labels': ['head', 'head', 'head'], 'boxes': array([[294.13876 , 125.354996, 337.02682 , 173.95854 ],
       [507.41782 , 121.629555, 557.50665 , 178.18756 ],
       [111.59824 , 134.82797 , 165.80174 , 195.49275 ]], dtype=float32)}


## Save image with given path

In [7]:
def save_image(dir_path, file_name, image):
    os.makedirs(dir_path, exist_ok=True)
    file_path = os.path.join(dir_path, file_name)

    cv.imwrite(file_path, image)

## Draw predict result

In [27]:
def draw_rect_with_result(input_image, result):
    # Modify area of nametag here
    relative_nametag_pos = [0, 10, 30, 20] #relative x0, y0, width, heigtht
    nametag_areas = []
    head_areas = []

    image = input_image.copy()
    
    for id, box in enumerate(result['boxes']):
        # Rectangel the area of human head
        x1, y1, x2, y2 = box.astype('int')
        head_pos = (x1, y1, x2, y2)
        head_areas.append(head_pos)
        cv.putText(image, str(id), (x1, y1), cv.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
        cv.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 3)

        # Rectagle the area of nametag
        cx = (x1 + x2) // 2
        dist_x, dist_y, width, height = relative_nametag_pos
        name_tag_pos = (cx + dist_x, y2 + dist_y, cx + dist_x + width, y2 + dist_y + height)
        nametag_areas.append(name_tag_pos)

        cv.putText(image, 'nametag', (name_tag_pos[0], name_tag_pos[1]-15), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)
        cv.rectangle(image, name_tag_pos[0:2], name_tag_pos[2:], (0, 0, 255), 1)

    # cv.imshow('Image', image)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    return image, head_areas, nametag_areas

def draw_rect(input_image, head_area=None, nametag_area=None):
    image = input_image.copy()
    if nametag_area:
        x1, y1, x2, y2 = nametag_area
        cv.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
        x1, y1, x2, y2 = head_area
        cv.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    elif head_area:
        x1, y1, x2, y2 = head_area
        cv.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    return image

In [96]:
# img_path = 'images\image0030.png'
# img_test = cv.imread(img_path)
# img_test = undistort_frame(img_test)
# middle_point = 270
# img_up, img_down = crop_image(img_test, middle_point)

# cv.imshow('Up', img_up)
# cv.imshow('Down', img_down)

# cv.waitKey(0)
# cv.destroyAllWindows()

# result1 = head_detection(img_up)
# result2 = head_detection(img_down)

In [20]:
def head_detection_images(input_path=None):
    if input_path is None:
        return

    if not os.path.exists(input_path):
        print('Directory not exist')
        return
    
    files = os.listdir(input_path)
    for i, file in enumerate(files):
        i_path = os.path.join(input_path, file)
        image = cv.imread(i_path)

        # Image Preprocess
        undistorted_image = undistort_frame(image)
        crop_undistorted_image_up, crop_undistorted_image_down = crop_image(undistorted_image, 270)

        # Head Detection
        result1 = head_detection(crop_undistorted_image_up)
        result2 = head_detection(crop_undistorted_image_down)

        # Show result
        crop_undistorted_image_up, _, nametag_areas1 = draw_rect_with_result(crop_undistorted_image_up, result1)
        crop_undistorted_image_down, _, nametag_areas2 = draw_rect_with_result(crop_undistorted_image_down, result2)

        # Concat image
        crop_undistorted_image_down = cv.rotate(crop_undistorted_image_down, cv.ROTATE_180)
        output_image = np.vstack([crop_undistorted_image_up, crop_undistorted_image_down])
        # # Coordinate transformation
        # vector_new = (crop_undistorted_image_down.shape[1], crop_undistorted_image_down.shape[0])
        # new_nametag_areas2 = []
        # for i, nametag_area2 in enumerate(nametag_areas2):
        #     new_x1 = vector_new[0] - nametag_area2[0]
        #     new_y1 = vector_new[1] - nametag_area2[1] + img_up.shape[0]
        #     new_x2 = vector_new[0] - nametag_area2[2]
        #     new_y2 = vector_new[1] - nametag_area2[3] + img_up.shape[0]

        #     new_nametag_areas2.append(new_x1, new_y1, new_x2, new_y2)
        

        # Save result
        # save_image('Head Detection Images', f'image_up{i}.jpg', crop_undistorted_image_up)
        # save_image('Head Detection Images', f'image_down{i}.jpg', crop_undistorted_image_down)
        # save_image("Head Detection Images", f'image{i}.jpg', output_image)


def head_detection_image(input_image):
    image = input_image.copy()
    # Image Preprocess
    crop_image_up, crop_image_down = crop_image(image, 270)

    # Head Detection
    result1 = head_detection(crop_image_up)
    result2 = head_detection(crop_image_down)

    # Show result
    crop_image_up, head_areas1, nametag_areas1 = draw_rect_with_result(crop_image_up, result1)
    crop_image_down, head_areas2, nametag_areas2 = draw_rect_with_result(crop_image_down, result2)

    # Concat image
    crop_image_down = cv.rotate(crop_image_down, cv.ROTATE_180)
    output_image = np.vstack([crop_image_up, crop_image_down])
    # Coordinate transformation
    vector_new = (crop_image_down.shape[1], crop_image_down.shape[0])
    new_head_areas2 = []
    new_nametag_areas2 = []
    for i, (head_area2, nametag_area2) in enumerate(zip(head_areas2, nametag_areas2)):
        new_x1 = vector_new[0] - head_area2[0]
        new_y1 = vector_new[1] - head_area2[1] + crop_image_up.shape[0]
        new_x2 = vector_new[0] - head_area2[2]
        new_y2 = vector_new[1] - head_area2[3] + crop_image_up.shape[0]

        new_head_areas2.append((new_x2, new_y2, new_x1, new_y1))

        new_x1 = vector_new[0] - nametag_area2[0]
        new_y1 = vector_new[1] - nametag_area2[1] + crop_image_up.shape[0]
        new_x2 = vector_new[0] - nametag_area2[2]
        new_y2 = vector_new[1] - nametag_area2[3] + crop_image_up.shape[0]

        new_nametag_areas2.append((new_x2, new_y2, new_x1, new_y1))

    return output_image, head_areas1 + new_head_areas2, nametag_areas1 + new_nametag_areas2



In [104]:
# Function test
img = cv.imread('images\image0030.png')
# Image Preprocess
undistorted_image = undistort_frame(img)

# Head Detection
output_image, head_areas, nametag_areas = head_detection_image(undistorted_image)

cv.imshow('Image', output_image)
cv.waitKey(0)
cv.destroyAllWindows()

In [87]:
for i, nametag_area in enumerate(nametag_areas):
    nametag_img = undistorted_image[nametag_area[1]:nametag_area[3], nametag_area[0]:nametag_area[2], :] 
    cv.imshow(f'Image{i}', nametag_img)
cv.imshow('Undistorted Image', undistorted_image)
cv.imshow('Image', output_image)

cv.waitKey(0)
cv.destroyAllWindows()

In [10]:
def nametag_detection(img, nametag_areas:np.array):
    # Showing the nametag area and edge detection result
    list_contours_x = []
    list_contours_y = []
    list_contours_xy = []
    is_nametag_list = []
    for idx, nametag_area in enumerate(nametag_areas):
        nametag_img = img[nametag_area[1]:nametag_area[3], nametag_area[0]:nametag_area[2]]
        # cv.imshow(f'NameTag{idx}', 
        #           cv.resize(nametag_img, (200, 150), interpolation=cv.INTER_AREA))
        # Convert to graycsale
        nametag_img_gray = cv.cvtColor(nametag_img, cv.COLOR_BGR2GRAY)
        cv.imwrite(f'SobelImages/SourceImage{idx}.jpg', nametag_img_gray)
        # sobelx = cv.resize(cv.Sobel(src=nametag_img_gray, ddepth=cv.CV_64F, dx=1, dy=0, ksize=3), (200, 150), interpolation=cv.INTER_AREA) # Sobel Edge Detection on the X axis
        # sobely = cv.resize(cv.Sobel(src=nametag_img_gray, ddepth=cv.CV_64F, dx=0, dy=1, ksize=3), (200, 150), interpolation=cv.INTER_AREA) # Sobel Edge Detection on the Y axis
        # sobelxy = cv.resize(cv.Sobel(src=nametag_img_gray, ddepth=cv.CV_64F, dx=1, dy=1, ksize=3), (200, 150), interpolation=cv.INTER_AREA)

        sobelx = cv.Sobel(src=nametag_img_gray, ddepth=cv.CV_64F, dx=1, dy=0, ksize=3)
        sobely = cv.Sobel(src=nametag_img_gray, ddepth=cv.CV_64F, dx=0, dy=1, ksize=3)
        sobelxy = cv.Sobel(src=nametag_img_gray, ddepth=cv.CV_64F, dx=1, dy=1, ksize=3)

        ret, sobelx = cv.threshold(sobelx, 100, 255, cv.THRESH_BINARY)
        ret, sobely = cv.threshold(sobely, 100, 255, cv.THRESH_BINARY)

        # Combine sobelx, sobely
        sobelxy = np.sqrt(np.square(sobelx) + np.square(sobely))
        # Normalization
        sobelxy *= 255 / sobelxy.max()
        sobelxy = sobelxy.astype(np.uint8)
        # cv.imshow("Magnitude", sobelxy)
        
        # cv.imshow(f'Sobel X{idx}', sobelx)
        # cv.imshow(f'Sobel Y{idx}', sobely)
        # cv.imshow(f'Sobel X Y using Sobel() function{idx}', sobelxy)

        # Draw contour of sobelx,y,xy
        sobelx = sobelx.astype(np.uint8)
        sobely = sobely.astype(np.uint8)
        sobelx_bgr = cv.cvtColor(sobelx, cv.COLOR_GRAY2BGR)
        sobely_bgr = cv.cvtColor(sobely, cv.COLOR_GRAY2BGR)
        sobelxy_bgr = cv.cvtColor(sobelxy, cv.COLOR_GRAY2BGR)

        contours_x, _ = cv.findContours(sobelx, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        list_contours_x.append(contours_x)
        for contour in contours_x:
            cv.drawContours(sobelx_bgr, [contour], -1, (0, 255, 0), 1)
        contours_y, _ = cv.findContours(sobely, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        list_contours_y.append(contours_y)
        for contour in contours_y:
            cv.drawContours(sobely_bgr, [contour], -1, (0, 255, 0), 1)
        contours_xy, _ = cv.findContours(sobelxy, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        list_contours_xy.append(contours_xy)
        for contour in contours_xy:
            cv.drawContours(sobelxy_bgr, [contour], -1, (0, 255, 0), 1)

        # Name tag prediction
        if len(contours_x) + len(contours_y) == 4 and len(contours_xy) == 1:
            is_nametag_list.append(True)
        else:
            is_nametag_list.append(False)

        # Storing sobel images for observation
        # dir_path = 'SobelImages'
        # file_name = 'SobelImage'
        # os.makedirs(dir_path, exist_ok=True)
        # path = os.path.join(dir_path, f'{file_name}')
        # # if idx == 6:
        # cv.imwrite(path+f'XY{idx}.jpg', sobelxy)
        # cv.imwrite(path+f'Y{idx}.jpg', sobely)
        # cv.imwrite(path+f'X{idx}.jpg', sobelx)
        # cv.imwrite(path+f'X_Contour{idx}.jpg', sobelx_bgr)
        # cv.imwrite(path+f'Y_Contour{idx}.jpg', sobely_bgr)
        # cv.imwrite(path+f'XY_Contour{idx}.jpg', sobelxy_bgr)

    return is_nametag_list

In [35]:
cap = cv.VideoCapture('sample.mp4')

while cap.isOpened():
    frame_idx = 0
    ret, frame = cap.read()
    if ret is None:
        break
    # Image Preprocess
    # undistorted_frame = undistort_frame(frame)

    # Head Detection
    output_image, head_areas, nametag_areas = head_detection_image(frame)

    # Nametag Detection
    is_nametag_list = nametag_detection(frame, nametag_areas)
    for i, is_nametag in enumerate(is_nametag_list):
        if is_nametag:
            # Showing nametag image
            # nametag_image = frame[nametag_areas[i][1]:nametag_areas[i][3], nametag_areas[i][0]:nametag_areas[i][2]]
            # nametag_image = cv.resize(nametag_image, (nametag_image.shape[1]*10, nametag_image.shape[0]*10), interpolation=cv.INTER_AREA)
            # cv.imshow(f'Nametag{i}', nametag_image)
            frame = draw_rect(frame, head_areas[i], nametag_areas[i])
            print(f'Frame{frame_idx}: X:{(head_areas[i][2] - head_areas[i][0]) // 2}, Y:{(head_areas[i][3] - head_areas[i][1]) // 2}')
        else:
            frame = draw_rect(frame, head_areas[i])
            
    cv.imshow('Frame', frame)
    # cv.imshow('Output', output_image)
    if cv.waitKey(30) == 27:
        break

    frame_idx += 1

cap.release()
cv.destroyAllWindows()


C:\Users\junee\AppData\Local\Temp\ipykernel_10784\1611432142.py:28: RuntimeWarning: divide by zero encountered in double_scalars
  sobelxy *= 255 / sobelxy.max()
C:\Users\junee\AppData\Local\Temp\ipykernel_10784\1611432142.py:28: RuntimeWarning: invalid value encountered in multiply
  sobelxy *= 255 / sobelxy.max()


Frame0: X:16, Y:18
Frame0: X:23, Y:19
Frame0: X:22, Y:24
Frame0: X:24, Y:25
Frame0: X:25, Y:29
Frame0: X:27, Y:29
Frame0: X:32, Y:32
Frame0: X:20, Y:17
